# Accessing Julich-Brain regions from injection sites coordinates

The injections sites coordinates were take directly from Nicola Palomero. The points were manually addressed via visual inspection.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import siibra

[siibra:INFO] Version: 1.0.1-alpha.1
[siibra:WARNING] This is a development release. Use at your own risk.
[siibra:INFO] Please file bugs and issues at https://github.com/FZJ-INM1-BDA/siibra-python.


## Loading data

Areas mapped by Rembrandt with their respective coordinates and regions.

In [2]:
data_regions = {
  "1": {
    "coord": [-55.2,-13.3,35.3],
    "region": "postcentral"
  },
  "2":{
    "coord": [-49.7,-21.3,30.3],
    "region": "postcentral"    
  },
  "3a,3b":{
    "coord": [-39.5,-8.9,33.1],
    "region": "postcentral"    
  },
  "4":{
    "coord":[-55.3,-4.5,28.3],
    "region":"precentral"
  },
  "5L": {
    "coord": [-20.4,-27.5,58.5],
    "region": "postcentral"
  },
  "5M": {
    "coord": [-2.7,-31.8,52.9],
    "region": "paracentral"
  },
  "6":{
    "coord": [-13.3,24.9,44.3],
    "region": "caudalmiddlefrontal"
  },
  "8":{
    "coord": [-23.1,22.7,29.5],
    "region":"superiorfrontal"
  },
  "9":{
    "coord": [-13.3,60.8,8.5],
    "region":"superiorfrontal"
  },
  "10L": {
    "coord": [-14,64.5,-29.6],
    "region": "frontalpole"
  },
  "10M":{
    "coord": [-1.1,56.2,-26.8],
    "region": "frontalpole"
  },
  "11":{
    "coord": [-0.7,23.7,-41.8],
    "region":"medialorbitofrontal"
  },
  "20":{
    "coord": [-59.6,-28.5,-40],
    "region": "inferiortemporal"
  },
  "21": {
    "coord": [-67.4,-14.7,-30.8],
    "region": "middletemporal"
  },
  "22": {
    "coord": [-63.5,-0.5,-20.9],
    "region": "superiortemporal"
  },
  "23":{
    "comment":"This is actually 23D, 23V is coming",
    "coord": [-1.1,-15.8,15.1],
    "region": "posteriorcingulate"
  },
  "24":{
    "coord":[-3.2,32.4,-11],
    "region": "caudalanteriorcingulate"
  },
  "31":{
    "coord": [-1.1,-25.6,24.6],
    "region": "posteriorcingulate"
  },
  "32":{
    "coord":[-0.4,38.2,-10.6],
    "region":"rostralanteriorcingulate"
  },
  "36":{
    "coord":[-37.6,-28.3,-43.6],
    "region":"fusiform"
  },
  "37B":{
    "coord":[-55,-56.8,-31.8],
    "region":"inferiortemporal"
  },
  "37M":{
    "coord":[-38,-44.5,-33.6],
    "region":"fusiform"
  },
  "37L": {
    "coord": [-64.3,-58.3,-17.8],
    "region": "inferiortemporal"
  },
  "38": {
    "coord": [-34.4,14.5,-51.1],
    "region": "temporalpole"
  },  
  "41": {
    "coord": [-46.7,-16.8,-2.8],
    "region": "transversetemporal"
  },
  "42": {
    "coord": [-57.7,-13.2,-3.2],
    "region": "superiortemporal",
    "comment": "deep into sulcus, lateral to 41"
  },
  "44": {
    "coord": [-58.6,18.8,-6.7],
    "region": "parsopercularis"
  },
  "45": {
    "coord": [-57.9,30.8,-18.5],
    "region": "parstriangularis"
  },
  "46": {
    "coord": [-43.3,52.5,-11.6],
    "region": "rostralmiddlefrontal"
  },
  "47": {
    "coord": [-50.9,30.6,-32.5],
    "region": "parsorbitalis"
  },
  "FG1":{
    "coord":[-26.6,-59.2,-21.5],
    "region":"fusiform"
  },
  "FG2":{
    "coord":[-40.6,-66.3,-24],
    "region":"fusiform"
  },
  "PFt": {
    "coord": [-65.6,-21,17],
    "region": "supramarginal"
  },
  "PFm": {
    "coord": [-53,-42.1,37.5],
    "region": "supramarginal"
  },
  "PGa": {
    "coord": [-42.5,-64.8,31.9],
    "region": "inferiorparietal"
  },
  "PGp": {
    "coord": [-33.3,-80.9,20.2],
    "region":  "inferiorparietal"
  },
  "V1":{
    "coord":[-6.9,-81.9,3.7],
    "region": "pericalcarine"
  },
  "V2d":{
    "coord":[-0.2,-82.3,12.7],
    "region":"cuneus"
  },
  "V2v":{
    "coord":[-0.2,-86.3,3.9],
    "region":"lingual"
  },
  "V3A":{
    "coord":[-5.4,-70.6,26.2],
    "region":"superiorparietal"
  },
  "V3d":{
    "coord":[-4.5,-81.3,24],
    "region":"superiorparietal"
  },
  "V3v":{
    "coord":[-21.4,-76.7,-21.1],
    "region":"lingual"
  },
  "V4v": {
    "coord": [-36.1,-88,-20.7],
    "region": "lateraloccipital"
  }
}

In [3]:
# Data from dict to Pandas Dataframe:
julichbrain2DK = pd.DataFrame.from_dict(data_regions, orient='index')

# Drop coordinates and remark columns:
julichbrain2DK = julichbrain2DK.drop(['comment'], axis=1)

# Convert index labels to a column. Then, rename it accordingly and set it as the new index.
julichbrain2DK = julichbrain2DK.reset_index().rename(columns={
    				'region': 'dk_RB', 
                    'index': 'julich_RB',
                    'coord': 'coordinates'}
					)

## Assigning the coordinates to brain regions

In [4]:
space = "mni colin 27"

with siibra.QUIET:  # suppress progress output
	julich_pmaps = siibra.get_map(
		parcellation="julich 3.1",
		space=space,
		maptype="statistical"
	)

	dk_pmaps = siibra.get_map(
		parcellation="desikan",
		space=space,
		maptype="labelled"
	)

In [5]:
coordinates = julichbrain2DK.coordinates.values
temp_regions = []

for coord in coordinates:
	point = siibra.Point(coord, space=space)
	with siibra.QUIET:  # suppress progress output
		assignments_julich = julich_pmaps.assign(point)
		assignments_dk = dk_pmaps.assign(point)

	# Check for empty assignments
	if assignments_julich.empty:
		print(f"Empty assignments for {coord} in Julich map")
		top_julich_region = None
	else:
		# Get the region with the highest map value for Julich
		top_julich_region = assignments_julich.loc[assignments_julich['map value'].idxmax()].region.name
		
	if assignments_dk.empty:
		print(f"Empty assignments for {coord} in DK map")
		top_dk_region = None
	else:
		# Get the region with the highest map value for DK
		top_dk_region = assignments_dk.loc[assignments_dk['map value'].idxmax()].region.name

	# Append data to array
	temp_regions.append([top_julich_region, top_dk_region, tuple(coord)])

# Convert array to DataFrame
temp_df = pd.DataFrame(temp_regions, columns=['julich_siibra', 'dk_siibra', 'coordinates'])

# Convert coordinates column to tuple in the original dataframe
julichbrain2DK['coordinates'] = julichbrain2DK['coordinates'].apply(tuple)

# Merge the new data with into new dataframe
julichbrain2DK_comparison = julichbrain2DK.merge(temp_df, on='coordinates')

# Reorder columns
julichbrain2DK_comparison = julichbrain2DK_comparison[['julich_RB', 'dk_RB', 'julich_siibra', 'dk_siibra', 'coordinates']]

Empty assignments for [-49.7, -21.3, 30.3] in DK map
Empty assignments for [-39.5, -8.9, 33.1] in DK map
Empty assignments for [-20.4, -27.5, 58.5] in DK map
Empty assignments for [-13.3, 24.9, 44.3] in DK map
Empty assignments for [-23.1, 22.7, 29.5] in DK map
Empty assignments for [-14, 64.5, -29.6] in DK map
Empty assignments for [-1.1, 56.2, -26.8] in DK map
Empty assignments for [-0.7, 23.7, -41.8] in DK map
Empty assignments for [-59.6, -28.5, -40] in DK map
Empty assignments for [-67.4, -14.7, -30.8] in DK map
Empty assignments for [-63.5, -0.5, -20.9] in DK map
Empty assignments for [-1.1, -15.8, 15.1] in DK map
Empty assignments for [-1.1, -25.6, 24.6] in DK map
Empty assignments for [-0.4, 38.2, -10.6] in DK map
Empty assignments for [-37.6, -28.3, -43.6] in DK map
Empty assignments for [-55, -56.8, -31.8] in DK map
Empty assignments for [-38, -44.5, -33.6] in DK map
Empty assignments for [-34.4, 14.5, -51.1] in DK map
Empty assignments for [-58.6, 18.8, -6.7] in DK map
Empty

In [6]:
julichbrain2DK_comparison

,julich_RB,dk_RB,julich_siibra,dk_siibra,coordinates
0,1,postcentral,Area 3b (PostCG) left,left precentral,"(-55.2, -13.3, 35.3)"
1,2,postcentral,Area PFt (IPL) left,None,"(-49.7, -21.3, 30.3)"
2,"3a,3b",postcentral,Area 6v3 (PreCG) left,None,"(-39.5, -8.9, 33.1)"
3,4,precentral,Area 3b (PostCG) left,left precentral,"(-55.3, -4.5, 28.3)"
4,5L,postcentral,Area 6d1 (PreCG) left,None,"(-20.4, -27.5, 58.5)"
5,5M,paracentral,Area 5M (SPL) left,left posterior cingulate,"(-2.7, -31.8, 52.9)"
6,6,caudalmiddlefrontal,Frontal-I.2 (GapMap) left,None,"(-13.3, 24.9, 44.3)"
7,8,superiorfrontal,Area MFG4 (MFG) right,None,"(-23.1, 22.7, 29.5)"
8,9,superiorfrontal,Area Fp1 (FPole) left,left superior frontal,"(-13.3, 60.8, 8.5)"
9,10L,frontalpole,Area MFG4 (MFG) right,None,"(-14, 64.5, -29.6)"
